<a href="https://colab.research.google.com/github/davidnoone/ExampleData/blob/main/Poisson_Jacobi_Spherical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Solving the Poisson equation in spherical (earth) coordinaes

The problem of interest is
$$
\zeta = \nabla^2 \psi
$$

Often $\zeta$ might be vorticity, and $\psi$ streamfunction. Or, divergence and velocity potential, also works this way.

Laplacian in spherical coordinates is:

$$
\nabla^2 \psi = \frac{1}{r^2}\left[
    \frac{1}{\cos^2 \theta} \frac{\partial^2 \psi}{\partial \lambda^2} +
        \frac{1}{\cos \theta} \frac{\partial}{\partial \theta} \left(
          \cos \theta \frac{\partial \psi}{\partial \theta}
          \right)
  \right]
$$

We note that $dx = r \cos \theta d\lambda$ and $dy = r d\theta$, which will come in handy. 

One can write the finite differences of this equation directly, however, one approach is to adjust the second term to find a "spherical corrrection" to the normal cartesian form. This is a simpler way to write the finite differences. 

The second term, with the product rule is. 

$$
        \frac{1}{\cos \theta} \frac{\partial}{\partial \theta} \left(
          \cos \theta \frac{\partial \psi}{\partial \theta}
          \right) = 
                  \frac{\partial^2 \psi}{\partial \theta^2} 
        + \frac{1}{\cos \theta} 
\frac{\partial \cos \theta }{\partial \theta} 
\left( \frac{\partial \psi}{\partial \theta} \right)
$$

Noting the definition of cos, the derivative is known. Thus the final form:

$$
\zeta = \nabla^2 \psi = 
    \frac{\partial^2 \psi}{\partial x^2(\theta)} +
    \frac{\partial^2 \psi}{\partial y^2}
       - \frac{\tan \theta}{r}  \frac{\partial \psi}{\partial y}
$$

This differes from the cartesion form in just two ways.
1. dx depends on laitude.
2. There is a correction term involving tan. 

Note that at the equator the tan term vanishes, and the system reduces to te cantesian form.

This can be expressed in finite differences, as usual.

$$
\zeta_{i,j} = 
    \frac{\psi_{i-1,j} - 2 \psi_{i,j} + \psi_{i+1,j}}{(\Delta x_j)^2} +
    \frac{\psi_{i,j-1} - 2 \psi_{i,j} + \psi_{i,j+1}}{(\Delta y)^2} 
       - \frac{\tan \theta_j}{2 r}  \frac{\psi_{i,j+1}-\psi_{i,j-1}}{\Delta y}
$$


To "invert" this as in the Jacobi method, one need to write move the term involving $\psi_{i,j}$ to the left, with all other terms remaining on the right.

The Jacobi iteration is:

$$
    \psi_{i,j}
 = \left( \frac{1}{\left( \frac{2}{(\Delta x_j)^2} +
                          \frac{2}{(\Delta y)^2} \right)}
   \right)
    \left[ \frac{\psi_{i-1,j} + \psi_{i+1,j}}{(\Delta x_j)^2} +
    \frac{\psi_{i,j-1} + \psi_{i,j+1}}{(\Delta y)^2} 
       - \frac{\tan \theta_j}{2 r}  \frac{\psi_{i,j+1}-\psi_{i,j-1}}{\Delta y}
       - \zeta_{i,j} \right]
$$





* DOUBLE CHECK MY MATH 





It is ever so slightly cleaner to multiply by $\Delta x$ and $\Delta y$ to give:

$$
    \psi_{i,j}
 = \frac{1}{2}\left( \frac{(\Delta x_j)^2   (\Delta y)^2) }
                          {(\Delta x_j)^2 + (\Delta y)^2) } 
              \right)
    \left[ \frac{\psi_{i-1,j} + \psi_{i+1,j}}{(\Delta x_j)^2} +
    \frac{\psi_{i,j-1} + \psi_{i,j+1}}{(\Delta y)^2} 
       - \frac{\tan \theta_j}{2 r}  \frac{\psi_{i,j+1}-\psi_{i,j-1}}{\Delta y}
       - \zeta_{i,j} \right]
$$

In [ ]:
#
# JACOBI METHOD: does full field EXCLUDING POLES.
#  (not successive, not over relaxation)
#
import numpy as np
import matplotlib.pyplot as plt

import time

def jacobi_spherical(psi_old, vor, dx, dy, lats, radius,poles=None):
    """ Jacobi iteration to update streamfunction estimate. 
      INPUT 
      psi_old[nlat,nlon]  last iteration (or first guess) of streafunction
      vor[nlat,nlon]      constraint/vorticity
      dy                  constant latitude spacing in units of meters (radias * dlat_radians)
      dx                  constant longitude spacing AT THE EQUATOR in units of meters (radias * dlon_radians)
      lats[nlat]          latitude values in degrres
      radius              radius of sphere. Probably earth. In metres.
      poles               [OPTIONAL] sets pole values as mean rather than zero

      RETURNS
      psi[nlat,nlon]      updated estimate of streamfunction

      Uses precomputed im, ic, ip as "i" indices to impliment
      black/red flop-flopping. Accounts for periodic boundary in longitude. 

      Note, nlon must be even, since we're using black/red symmetry.

    """

    # precompute indices (could be global)
    (ny,nx) = np.shape(vor)

    ic_all = np.arange(nlon)      # all points
    im_all = np.roll(ic_all,+1)   # wrap at low end
    ip_all = np.roll(ic_all,-1)   # wrap at high end

    # Create a copy of psi to store the updated values
    psi = psi_old.copy()

    for ibr in [0,1]:             # "black"/red flag
      ic = ic_all[ibr::2]         
      ip = ip_all[ibr::2]
      im = im_all[ibr::2]

      for j in range(1, ny - 1):    # exclude ends/boundary values
          jc = np.repeat(j,nx/2)

          theta = np.radians(lats[j])          # latitude in radians

          coslat = np.cos(theta)
          tanlat = np.tan(theta)

          dx_lat  = dx*coslat                   # "dx" in meters for FOR THIS LATITUDE

          rdxsq = 1./(dx_lat*dx_lat)            # reciprocal of dx^2 FOR THIS LATITUDE
          rdysq = 1./(dy*dy)                    # reciprocal of dy^2 (same for all latitudes)
          rden = 0.5/(rdxsq + rdysq)            # reciprocal of denominator

          dpsi_x2  = rdxsq*(psi[jc  , ip] + psi[jc  , ip])    # the "x" bit
          dpsi_y2  = rdysq*(psi[jc+1, ic] + psi[jc-1, ic])    # the "y" bit

          sphere_correction = tanlat * (psi[j+1, ic] - psi[j-1, ic])/(2*radius*dy)

          psi[jc, ic] = rden * ( dpsi_x2 + dpsi_y2 - sphere_correction - vor[jc,ic])

      # Special case of poles, as a boundary condition.
      if poles is not None:
        psi[ 0, :] = np.mean(psi[ 1, :])
        psi[-1, :] = np.mean(psi[-2, :])


    return psi

def solve_poisson_jacobi_spherical(vor,dx,dy,lats,radius):

  # Define the convergence criteria, and iterate
  maxiter = 100
  max_error = 1e-6

  psi = np.zeros_like(vor)  # first guess: Sets pole points=0 as boundary condition

  for iteration in range(maxiter):
      # Compute the new value of psi: requires dx=dy!
      psi_new = jacobi_spherical(psi, vor, dx, dy, lats, radius)      # neeed to pass latitudes

      # Check for convergence
      error = np.max(np.abs(psi_new - psi))
      if error < max_error:
          break

      # Update psi with the new values
      psi = psi_new

  print('Converged after ',iteration,' iterations')
  return psi



#------------------------------------------------
#
# Radius of earth (your sphere), in meters   
rearth = 6_378_100        

# Create some fake data (as if it might come from NetCDF)
nlon = 360
nlat = 181
lons = np.linspace(0,360,num=nlon)
lats = np.linspace(-90,90,num=nlat,endpoint=True)
vort = np.random.random((nlat,nlon))
print('VORT shape:',np.shape(vort))


# Grid spacing in units of m
dx = 2*np.pi*rearth/ (nlon)         # At the equator
dy =   np.pi*rearth/ (nlat - 1)     # Including poles

# Solve the poisson problem
t0 = time.process_time()

psi = solve_poisson_jacobi_spherical(vort,dx,dy,lats,rearth)

delt = time.process_time()-t0
print('Jacobi solver in',delt*1000,'msec')



# Plot the solution
#%
fig, ax = plt.subplots()
cs = ax.contourf(lons, lats, psi, cmap='Spectral')
ax.set_title('Streamfunction (Jacobi, T='+str(int(delt*1000))+' msec)')
plt.colorbar(cs)
ax.set_aspect('equal')
plt.show()
